In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import optim, nn
import torch.nn.functional as F 

from torchvision import transforms

from torch.utils.data import Dataset, DataLoader, random_split

In [32]:
torch.manual_seed(0)

In [33]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
path = "drive/MyDrive/Colab Notebooks/fashion-mnist.csv"
fashion_mnist_csv = pd.read_csv(path)

In [35]:
class FashionMNISTDataset(Dataset):
    def __init__(self, data, transform = None):
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        for i in self.fashion_MNIST:
            label.append(i[0])
            image.append(i[1:])
        self.labels = np.asarray(label)
        self.images = np.asarray(image)/255

    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

In [36]:
TRAIN_SIZE = 0.8 
TEST_SIZE = 1 - TRAIN_SIZE
BATCH_SIZE = 64

dataset = FashionMNISTDataset(fashion_mnist_csv)

TRAIN_DATASET_SIZE = int(len(dataset)*TRAIN_SIZE) 
TEST_DATASET_SIZE = len(dataset) - TRAIN_DATASET_SIZE
train_set, test_set = random_split(dataset = dataset, lengths= [TRAIN_DATASET_SIZE,TEST_DATASET_SIZE])

train_loader = DataLoader(train_set, BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, BATCH_SIZE)

In [43]:
class FashionMNISTNN(nn.Module):
    def __init__(self, input_dim, h1,h2, output_dim):
        super().__init__()

        self.fc1 = nn.Linear(input_dim, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.fc3 = nn.Linear(h2, output_dim)

        self.dropout = nn.Dropout(0.1)

    def forward(self, x):                           
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = F.softmax(self.fc3(x),dim=1)                  

        return x

In [38]:
def calc_accuracy(a, y):                            
    a = torch.max(a, dim=1)[1]
    assert not torch.any(torch.isnan(a))
    return torch.mean((a == y).float()).item()

In [44]:
model = FashionMNISTNN(784,128,32,10)

optimizer = optim.Adam(model.parameters(), lr=float(0.01))

In [45]:
num_epochs = 25
loss_history = []
accuracy_history = []

for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for image, label in train_loader:
        image = image.view(-1, 784)
        label = label.view(-1)
        z = model(image.float())
        loss = F.cross_entropy(z, label)
        accuracy = calc_accuracy(z, label)

        optimizer.zero_grad()      
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() / TRAIN_DATASET_SIZE * BATCH_SIZE
        epoch_accuracy += accuracy / TRAIN_DATASET_SIZE * BATCH_SIZE
    
    loss_history.append(epoch_loss)
    accuracy_history.append(epoch_accuracy)

    if epoch % 5 == 0:
        print("Epoch: {}, Loss: {}, Accuracy: {}".format(epoch, round(epoch_loss, 3), round(100*epoch_accuracy, 3)))

Epoch: 0, Loss: 1.875, Accuracy: 58.45
Epoch: 5, Loss: 1.7, Accuracy: 76.038
Epoch: 10, Loss: 1.685, Accuracy: 77.637
Epoch: 15, Loss: 1.709, Accuracy: 75.225
Epoch: 20, Loss: 1.691, Accuracy: 77.0


In [46]:
num_epochs = 5
avg_loss = 0
avg_accuracy = 0

for epoch in range(num_epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for image, label in test_loader:
        z = model(image.float())
        loss = F.cross_entropy(z, label)
        accuracy = calc_accuracy(z, label)

        epoch_loss += loss.item() / TEST_DATASET_SIZE * BATCH_SIZE
        epoch_accuracy += accuracy / TEST_DATASET_SIZE * BATCH_SIZE

    avg_loss += epoch_loss / num_epochs
    avg_accuracy += epoch_accuracy / num_epochs

print("Number of Epochs tested: {}, Average loss: {}, Average Accuracy: {}".format(num_epochs, round(avg_loss, 3), round(100*avg_accuracy, 3)))

Number of Epochs tested: 5, Average loss: 1.764, Average Accuracy: 75.61
